In [54]:
import numpy as np
import pandas as pd; pd.set_option('display.max_rows', 10000)
import allel 
import matplotlib.pyplot as plt
import zarr
import h5py
import seaborn as sns
from sklearn import metrics
from tqdm import tqdm
#from mpl_toolkits.basemap import Basemap

In [3]:
%run "~/ag1000g/selective_sweeps/scripts/sweeps_functions.py"
samples = pd.read_csv("../../data/samples.meta.txt", sep='\t')

### Ne estimates from Ag1000g populations (LDNe)

Estimates of effective population size were implemented in NeEstimator v2.1 (Do *et al*., 2014), using the Linkage Disequilibrium method (LDNe) of Waples and Do (2008). This method is widely used, and generally performs robustly in comparisons between single-sample estimators (see references for further reading).

In [65]:
Ne = pd.read_csv("Ne_analyses.LDNe", sep="\t")

In [73]:
Ne[Ne['AF'] == 'minAF_0.05']

,AF,chrom,pop,sample_size,independent_comparisons,overall_r^2,expected_r^2,Ne_estimate,Parametric_CI_lower,Parametric_CI_upper,Jackknife_CI_lower,Jackknife_CI_upper
0,minAF_0.05,3L,GHcol,55.0,6465956,0.019633,0.019242,851.2,806.9,900.7,338.5,Infinite
4,minAF_0.05,3R,GHcol,55.0,6705702,0.019729,0.019240,680.4,652.1,711.2,224.3,Infinite
8,minAF_0.05,3L,GHgam,12.0,13844967,0.109337,0.108240,279.1,259.7,301.6,98.9,Infinite
12,minAF_0.05,3R,GHgam,12.0,13760829,0.110819,0.108240,117.7,114.0,121.7,34.1,Infinite
16,minAF_0.05,3L,BFgam,92.0,2357246,0.011382,0.011249,2503.3,2167.7,2960.1,1317.0,21833.9
20,minAF_0.05,3R,BFgam,92.0,2520667,0.011443,0.011247,1699.6,1541.9,1892.6,1040.1,4508.3
24,minAF_0.05,3L,BFcol,75.0,3364929,0.013900,0.013903,Infinite,17897.2,Infinite,10892.7,Infinite
28,minAF_0.05,3R,BFcol,75.0,3173472,0.013897,0.013902,Infinite,19659.2,Infinite,9299.5,Infinite
32,minAF_0.05,3L,UGgam,112.0,2733796,0.009316,0.009184,2515.7,2249.7,2851.9,1960.6,3500.0
36,minAF_0.05,3R,UGgam,112.0,2759529,0.009329,0.009184,2304.8,2080.2,2583.0,1798.6,3198.8


Be aware that both methods to produce confidence intervals, the parametric and pseudo-jackknife method, may be suboptimal (Jones *et al*., 2016), unfortunately, the amendments they suggest have not yet been implemented.

### estimating Ne from theta=4Nemu

In [59]:
pops = samples.population.unique()
chroms = ['3L', '3R']

mu=3.5e-9

Ne = dict()
Ne_Ag = dict()

Ag_array  = zarr.open_array(f"/home/sanj/ag1000g/data/ag1000g.phase2.ar1.pass/{chrom}/calldata/GT/")
pos  = zarr.open_array(f"/home/sanj/ag1000g/data/ag1000g.phase2.ar1.pass/{chrom}/variants/POS")
print("-------------------  Arrays loaded ------------------------")
geno = allel.GenotypeChunkedArray(Ag_array)
        
for pop in pops:
    for chrom in chroms:
        
        pop_bool = samples.population == pop
        pop_geno = geno.compress(pop_bool, axis=1)
        
        print(f"Counting alleles {pop} {chrom}")
        ac = pop_geno.count_alleles()
        print("Computing theta")
        theta = allel.watterson_theta(pos, ac)
        print('done')
        Neff = theta/(4*mu)
        
        Ne[chrom] = Neff
    
    Ne_Ag[pop] = dict(Ne)



  0%|          | 0/16 [00:00<?, ?it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

-------------------  Arrays loaded ------------------------
Counting alleles GHcol 3L
Computing theta





 50%|█████     | 1/2 [00:55<00:55, 55.29s/it]

done
Counting alleles GHcol 3R
Computing theta





100%|██████████| 2/2 [01:55<00:00, 57.91s/it]


  6%|▋         | 1/16 [01:55<28:57, 115.83s/it]


  0%|          | 0/2 [00:00<?, ?it/s]

...

done
Counting alleles CIcol 3L
Computing theta





 50%|█████     | 1/2 [01:36<01:36, 96.00s/it]

done
Counting alleles CIcol 3R
Computing theta





100%|██████████| 2/2 [03:08<00:00, 94.13s/it]


100%|██████████| 16/16 [42:10<00:00, 158.16s/it]

done


In [76]:
Ne_theta = pd.DataFrame.from_dict(Ne_Ag).T
Ne_theta.round()

,3L,3R
GHcol,952726.0,952726.0
GHgam,816605.0,816605.0
BFgam,1397081.0,1397081.0
BFcol,1242037.0,1242037.0
UGgam,1321118.0,1321118.0
GM,945927.0,945927.0
GW,1259574.0,1259574.0
KE,237080.0,237080.0
CMgam,1767792.0,1767792.0
FRgam,321054.0,321054.0


### References